In [ ]:
# load_model, tokenizer
from transformers import AutoModelForCausalLM
from transformers.models.gpt_neox import GPTNeoXTokenizerFast
from time import time

import torch
import os
import numpy as np

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="5"
os.environ["CUDA_MODULE_LOADING"]="LAZY"

base_model ="nlpai-lab/kullm-polyglot-12.8b-v2"
model = AutoModelForCausalLM.from_pretrained(
base_model,
torch_dtype=torch.float16,
low_cpu_mem_usage=True,
).cuda()

tokenizer = GPTNeoXTokenizerFast.from_pretrained('nlpai-lab/kullm-polyglot-12.8b-v2', export=True)


In [ ]:
# save model
model.save_pretrained("/home/team_ai/JMJ/nlp_model/kullm_dev/HS/sav_bin2")

# after this, run optimum_cli export
# $ optimum-cli export onnx --model <saved_bin_folder_pth> --task text-generation <folder_path_for_sav_ONNX>
os.system("optimum-cli export onnx --model /home/team_ai/JMJ/nlp_model/kullm_dev/HS/sav_bin2 --task text-generation /home/team_ai/JMJ/nlp_model/kullm_dev/HS/sav_onnx2")

In [ ]:
#model_test
token = tokenizer("black top big t-shirts billie eilish", return_tensors="pt")
r = model(token.input_ids.cuda())
output = tokenizer.batch_decode(np.argmax(r[0].detach().cpu().numpy(),2), skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(output)

for i in range(20):
    t = time()
    token = tokenizer("black top big t-shirts billie eilish", return_tensors="pt")
    r = model(token.input_ids.cuda())
    output = tokenizer.batch_decode(np.argmax(r[0].detach().cpu().numpy(),2), skip_special_tokens=True, clean_up_tokenization_spaces=False)
    print(time()-t)

In [7]:
# onnx_test
from transformers.models.gpt_neox import GPTNeoXTokenizerFast
from onnxruntime import InferenceSession
import numpy as np
providers =  ['CUDAExecutionProvider', 'CPUExecutionProvider']
tokenizer = GPTNeoXTokenizerFast.from_pretrained('nlpai-lab/kullm-polyglot-12.8b-v2', export=True)
session = InferenceSession("/home/team_ai/JMJ/nlp_model/kullm_dev/HS/sav_onnx2/decoder_model.onnx", providers=providers)



The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'GPTNeoXTokenizerFast'.
2023-09-05 06:03:47.234398815 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:614 CreateExecutionProviderInstance] Failed to create TensorrtExecutionProvider. Please reference https://onnxruntime.ai/docs/execution-providers/TensorRT-ExecutionProvider.html#requirements to ensure all dependencies are met.
2023-09-05 06:03:47.234435926 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:640 CreateExecutionProviderInstance] Failed to create CUDAExecutionProvider. Please reference https://onnxruntime.ai/docs/execution-providers/CUDA-ExecutionProvider.html#requirements to ensure all dependencies are met.


In [8]:
# ONNX Runtime expects NumPy arrays as input
inputs = tokenizer("black top big t-shirts billie eilish", return_tensors="np")
outputs = session.run(output_names=["logits"],input_feed=dict(inputs))
output = tokenizer.batch_decode(np.argmax(outputs[0],2), skip_special_tokens=True, clean_up_tokenization_spaces=False)
output

['abing.ea hut bopshirts,igb eilish b']